### Final Propensity matching results

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [4]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

### Load pre matching data

In [5]:
%%time
pp_str = 'Pre_Matching_DATA/'

treated2 = pd.read_pickle(pp_str+'treated2.p')
controls = pd.read_pickle(pp_str+'controls.p')
pmpm =pd.read_pickle(pp_str+'pmpm.p')
random_controls = pd.read_pickle('tempHybridSelectionMoreData/random.p')

CPU times: user 6.91 s, sys: 14.9 s, total: 21.8 s
Wall time: 21.8 s


### Pick Columns for matching

In [6]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match =  ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY', 
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6','LOB']

all_columns1 = list(set(col for col in pmpm.columns)-set(['PERS_ID', 'MYR','MEDICAID_LOB',
                                                         'IP_BIN', 'ER_BIN', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C','IP_BIN2', 'ER_BIN2']))

In [7]:
%%time

propensity_func_args = {'match_on':all_columns1, 'neighbors_per_treated': 5, 'onset': "CC_DIABETES", "onset_val" : 0}
matched0, matched_data0 = matched_population(treated2.copy(), 
                                           controls.copy(),
                                           propensity_func, 
                                           propensity_func_args)




Mathcing Function: propensity_func
Propensity Score Matching...fitting model
Fitted: LogisticRegression
Creating Matched Set...


0it [00:00, ?it/s]

iter: 0


200it [11:57,  3.60s/it]

iter: 200


...

iter: 1200


1400it [1:24:17,  3.72s/it]

iter: 1400


1565it [1:34:22,  3.62s/it]


CPU times: user 1h 41min 19s, sys: 41min 23s, total: 2h 22min 43s
Wall time: 1h 40min 39s


####  In addition to the previously created matched groups create a random group

In [8]:
Path('./tempPropALL').mkdir(exist_ok = True)
p = './tempPropALL/'
matched_data0.to_pickle(p+'match0.p')
treated2.to_pickle(p+'treated0.p')

In [9]:
pmpm2 = pmpm
try:
    pmpm2.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
except:
    pass

In [10]:
from utils import statistics_first_month, p_val, stat_matrices 

### List containing all numerical columns

In [11]:
all_columns = list(set(col for col in pmpm.columns)-set(['PERS_ID', 'MYR','MEDICAID_LOB']))

### Stats First Month(Treated and Matched Controls)

In [12]:
%%time
tr_desc0, treated_first_month = statistics_first_month(treated2, all_columns, return_array = True)
con_desc0, controls_first_month = statistics_first_month(matched_data0, all_columns, return_array = True)

CPU times: user 3.87 s, sys: 41 ms, total: 3.91 s
Wall time: 3.91 s


### Stats First Month(General Pop and Random)

In [13]:
%%time
pmpm_desc = statistics_first_month(pmpm2, all_columns)
random_desc = statistics_first_month(random_controls, all_columns)

CPU times: user 2min 30s, sys: 7.68 s, total: 2min 38s
Wall time: 2min 38s


In [14]:
%%time
conds_first, full_first = stat_matrices(tr_desc0, con_desc0, pmpm_desc, random_desc, columns = columns_to_match)
conds_firstAllCols, full_firstAllCols = stat_matrices(tr_desc0, con_desc0, pmpm_desc, random_desc, columns = all_columns)

CPU times: user 38.1 ms, sys: 976 µs, total: 39 ms
Wall time: 38 ms


## Statistics on matching date

In [15]:
conds_first.round(3)

Treated Matched Controls Test-TC        General Population   Random
                    mean             mean  t-stat   pval               mean     mean
AGE_AT_MIDMONTH   50.206           49.180   2.156  0.031             44.311   47.606
ALLOW_AMT        693.750          710.127  -0.201  0.841           1065.926  648.803
MBR_GNDR           0.206            0.382 -13.602  0.000              0.443    0.431
CC_TOBACCO         0.061            0.100  -4.837  0.000              0.074    0.098
CC_PRESSURE        0.001            0.004  -1.772  0.076              0.007    0.007
CC_OBESITY         0.456            0.261  15.790  0.000              0.109    0.228
CC_HYPERTENSION    0.325            0.260   5.341  0.000              0.208    0.249
CC_HYPOTHYROID     0.095            0.063   4.662  0.000              0.047    0.063
CC_DX_CNT          2.706            2.335   6.287  0.000              1.824    2.344
ACUTE2             0.039            0.049  -1.709  0.088              0.063    0.052
ACUTE6             0.112            0.144  -2.675  0.007              0.063    0.153
IP_BIN6            0.028            0.037  -1.455  0.146              0.021    0.047
ER_BIN6            0.089            0.115  -2.460  0.014              0.047    0.116
LOB                0.952            0.821  13.263  0.000              0.756    0.788

In [16]:
full_first

Treated                    Matched Controls                       Test-TC               General Population                         Test-TG                    Random                       Test-TR              
                       mean          std count             mean          std count     t-stat          pval               mean          std   count     t-stat          pval        mean          std count     t-stat          pval
AGE_AT_MIDMONTH   50.205736    12.733903  1604        49.179864    18.138293  7628   2.156140  3.109853e-02          44.310976    20.053585  356446  11.762030  6.207968e-32   47.606399    19.811819  7876   5.046484  4.583703e-07
ALLOW_AMT        693.750000  2216.211182  1604       710.126901  3106.957712  7628  -0.200640  8.409842e-01        1065.926022  6933.418650  356446  -2.149328  3.160904e-02  648.802692  3166.090228  7876   0.542150  5.877276e-01
MBR_GNDR           0.205736     0.404364  1604         0.382407     0.486007  7628 -13.602254  9.791071e-42           0.442749     0.496712  356446 -19.081951  3.914644e-81    0.431056     0.495255  7876 -17.097185  1.439342e-64
CC_TOBACCO         0.061097     0.239583  1604         0.099633     0.299530  7628  -4.837389  1.336844e-06           0.074202     0.262100  356446  -1.998705  4.564109e-02    0.098400     0.297874  7876  -4.714462  2.458070e-06
CC_PRESSURE        0.001247     0.035300  1604         0.004195     0.064638  7628  -1.771949  7.643599e-02           0.007390     0.085645  356446  -2.871419  4.086576e-03    0.006602     0.080991  7876  -2.598386  9.380812e-03
CC_OBESITY         0.456359     0.498247  1604         0.261143     0.439287  7628  15.790076  1.929192e-55           0.108841     0.311440  356446  44.434357  0.000000e+00    0.228035     0.419592  7876  19.209513  1.041149e-80
CC_HYPERTENSION    0.324813     0.468451  1604         0.259701     0.438499  7628   5.340566  9.486088e-08           0.207717     0.405674  356446  11.525694  9.915163e-31    0.249365     0.432673  7876   6.274847  3.652926e-10
CC_HYPOTHYROID     0.094763     0.292979  1604         0.062533     0.242137  7628   4.661582  3.181886e-06           0.047300     0.212281  356446   8.916454  4.836392e-19    0.063484     0.243847  7876   4.516264  6.370084e-06
CC_DX_CNT          2.706359     2.227360  1604         2.334688     2.136135  7628   6.286727  3.389153e-10           1.824419     2.412274  356446  14.614465  2.345498e-48    2.343702     2.439688  7876   5.504457  3.799632e-08
ACUTE2             0.038653     0.208377  1604         0.049423     0.233648  7628  -1.708691  8.754191e-02           0.063258     0.243426  356446  -4.041413  5.314111e-05    0.051549     0.237733  7876  -2.020160  4.339483e-02
ACUTE6             0.111596     0.412699  1604         0.144206     0.450132  7628  -2.674612  7.494766e-03           0.063258     0.243426  356446   7.901966  2.753095e-15    0.153377     0.475791  7876  -3.274968  1.060534e-03
IP_BIN6            0.028055     0.199403  1604         0.036969     0.227739  7628  -1.454751  1.457721e-01           0.020587     0.141996  356446   2.097153  3.598074e-02    0.046851     0.263725  7876  -2.701482  6.915408e-03
ER_BIN6            0.088529     0.359723  1604         0.114840     0.395232  7628  -2.460491  1.389287e-02           0.047286     0.212251  356446   7.732311  1.058846e-14    0.116176     0.402957  7876  -2.548765  1.082613e-02
LOB                0.951995     0.213843  1604         0.820923     0.383442  7628  13.263072  8.754153e-40           0.755775     0.429627  356446  18.281467  1.257397e-74    0.787836     0.408866  7876  15.649481  1.606802e-54

### Statistics Over all dates (treated, controls)

In [17]:
%%time
tr_desc0A = treated2[all_columns].describe().loc[['mean', 'std', 'count']].T
tr_desc0A['count'] = tr_desc0A['count'].astype(int)

con_desc0A = matched_data0[all_columns].describe().loc[['mean', 'std', 'count']].T
con_desc0A['count'] = con_desc0A['count'].astype(int)

CPU times: user 722 ms, sys: 0 ns, total: 722 ms
Wall time: 722 ms


### Statistics Over all dates (general population, Random)

In [18]:
%%time
pmpm_descA = pmpm2[all_columns].describe().loc[['mean', 'std', 'count']].T
pmpm_descA['count'] = pmpm_descA['count'].astype(int)

random_descA = random_controls[all_columns].describe().loc[['mean', 'std', 'count']].T
random_descA['count'] = random_descA['count'].astype(int)

CPU times: user 28.4 s, sys: 2.29 s, total: 30.7 s
Wall time: 30.6 s


In [19]:
%%time
conds, full = stat_matrices(tr_desc0A, con_desc0A, pmpm_descA, random_descA, columns = columns_to_match)
conds_AllCols, full_AllCols = stat_matrices(tr_desc0A, con_desc0A, pmpm_descA, random_descA, columns = all_columns)

CPU times: user 29.4 ms, sys: 26 µs, total: 29.4 ms
Wall time: 27.9 ms


In [20]:
conds.round(3)

Treated Matched Controls Test-TC        General Population   Random
                    mean             mean  t-stat   pval               mean     mean
AGE_AT_MIDMONTH   50.837           51.306  -4.057  0.000             50.379   50.236
ALLOW_AMT        782.435          735.910   1.742  0.081            857.856  706.609
MBR_GNDR           0.189            0.373 -57.069  0.000              0.433    0.429
CC_TOBACCO         0.063            0.102 -19.046  0.000              0.102    0.100
CC_PRESSURE        0.002            0.006  -7.172  0.000              0.013    0.008
CC_OBESITY         0.505            0.302  63.048  0.000              0.255    0.286
CC_HYPERTENSION    0.325            0.301   7.632  0.000              0.329    0.300
CC_HYPOTHYROID     0.095            0.073  11.848  0.000              0.078    0.075
CC_DX_CNT          2.918            2.648  17.215  0.000              2.955    2.759
ACUTE2             0.032            0.045  -8.707  0.000              0.060    0.050
ACUTE6             0.100            0.136 -12.026  0.000              0.168    0.150
IP_BIN6            0.025            0.040  -9.687  0.000              0.055    0.049
ER_BIN6            0.079            0.102  -9.099  0.000              0.122    0.111
LOB                0.953            0.832  50.532  0.000              0.802    0.803

#### Distribution Plots For Treated and Matched Controls
1)  Sick(S) Treated versus  Sick Control groups  
2) Healthy(H) Treated versus Healthy Control Groups

### Matrices and Figures for the paper

In [21]:
columns_to_match

['AGE_AT_MIDMONTH',
 'ALLOW_AMT',
 'MBR_GNDR',
 'CC_TOBACCO',
 'CC_PRESSURE',
 'CC_OBESITY',
 'CC_HYPERTENSION',
 'CC_HYPOTHYROID',
 'CC_DX_CNT',
 'ACUTE2',
 'ACUTE6',
 'IP_BIN6',
 'ER_BIN6',
 'LOB']

In [22]:
mapping = dict(zip(columns_to_match, ['Age', 'Total Cost', 'Gender', 'Tobacco Use', 'Pressure', 'Obesity', 
                                     'Hypertension', 'Hypothyroid', 'Disease Count','Acute Care 2 Months', 'Acute Care 6 Months',
                                      'Inpatient Visits 6 Months', 'Emergency Vistits 6 Months', 'Line of Bussiness']))
mapping2 = dict(zip(all_columns, all_columns))
#mapping = dict(zip(columns_to_match, columns_to_match))
new_names = list(mapping.values())

In [23]:
base_path = './results/propensity_ALL/'
p = Path(base_path)
p.mkdir(parents = True, exist_ok = True)

In [24]:
import matplotlib as mpl
sns.set_context(context = 'paper')
mpl.style.use('seaborn-paper')

In [25]:
from diabetes_onset_plotter import plot_hist, save_arrays, save_arrays2


#### remove %%capture to see output

In [26]:
%%capture

dist_plot_args = {'hist':True}
axes = plot_hist(treated_first_month.copy(), 
                 controls_first_month.copy(), mapping2, ['ALLOW_AMT'] , 
                 data3 = None, 
                 save_path = base_path+'figures/distributions_all/',
                dist_plot_args = dist_plot_args)

In [27]:
%%capture
dist_plot_args = {'hist':True}
axes = plot_hist(treated_first_month.copy(), 
                 controls_first_month.copy(), mapping, ['ALLOW_AMT'] , 
                 data3 = None, 
                 save_path = base_path+'figures/distributions_selection/',
                dist_plot_args = dist_plot_args)

### Save enhanced stats arrays(these will be used at the end)
Arrays with: Statistics in first day, statistics  in the span of monthsm statistiocs from selected columns and all columns

In [28]:
arrays = [conds_first, full_first, conds_firstAllCols, full_firstAllCols,
         conds, full, conds_AllCols, full_AllCols]
names = ['conds_first', 'full_first', 'conds_firstAllCols', 'full_firstAllCols',
         'conds', 'full', 'conds_AllCols', 'full_AllCols']
save_arrays2(arrays, names, base_path+'tables/stats/')